In [1]:
!pip install langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9a21ee6577b7e99993a5324d310fe3af1b277a35564d9c6ef4efb5edcde614e0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import json
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

from tqdm import tqdm
import numpy as np
import json

from torch.nn import functional as F


In [203]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [270]:
with open("/content/drive/MyDrive/valid_abstracts2013.json", 'r') as f:
    data = json.load(f)



In [206]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [207]:
embed_model_id = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(embed_model_id)

In [208]:
splitter = SentenceTransformersTokenTextSplitter(
    model_name=embed_model_id,  # specify the model used for tokenization
    chunk_overlap=10,  # set the overlap between consecutive text chunks
)
print(splitter.maximum_tokens_per_chunk)

384


In [211]:
for key, value in data.items():
    print(f"Key: {key}")

Key: abstracts
Key: authior
Key: url
Key: title
Key: article_id


In [212]:
# initialize an empty list to store vector data information
vector_data = []

# loop over each document in the "train" split of the dataset
for i in range(len(data['abstracts'])):


    # split abstract text into chunks
    chunks = splitter.split_text(text=data['abstracts'][i])

    # process each chunk
    for j, chunk in enumerate(chunks):

        # create metadata dictionary
        metadata = {
            "article_id": data['article_id'][i][0][0],
            "title": data["title"][i],
            "authior":  ', '.join([author[0] for author in data["authior"][i]]) ,

            "url": data["url"][i],
            "text_chunk_id": j,
            "text": chunk,
        }

        # embed the chunk using sentence transformer model
        embedding = model.encode(chunk).tolist()

        # generate unique identifier
        ids = f"{data['article_id'][i][0][0]}_{j}"

        # append data to the vector_data list
        vector_data.append((ids, embedding, metadata))


In [ ]:
for i in range(len(vector_data)):
  vector_data[i][2]["authior"]=vector_data[i][2]["authior"].split(", ")



In [215]:
!pip install -qU pinecone-client==2.2.4

In [216]:
# import libraries
import pinecone
import time

In [217]:
import os

In [218]:
# initialize Pinecone with API key and environment
# get API key and environment from app.pinecone.io console
pinecone.init(
    api_key="ba161cc5-2ff9-41f9-9754-cb0e319c6703",
    environment= "gcp-starter")

# get index name from environment variables
index_name = os.getenv('PC_INDEX')

In [219]:
index=pinecone.Index("database")

In [220]:
def insert_pinecone(index, vector_data):

    batch_size = 200

    # loop over the data in batches for efficient upsert
    for i in tqdm(range(0, len(vector_data), batch_size)):
        # get the current batch of data
        code = [vector_data[j] for j in range(i, min(i + batch_size, len(vector_data)))]

        # upsert the batch of data into the Pinecone index
        index.upsert(vectors=code)

        # sleep for 1 second to avoid API rate limits
        time.sleep(1)

In [262]:
# check if we have vector data to insert
if vector_data:
    insert_pinecone(index, vector_data)

100%|██████████| 6/6 [00:12<00:00,  2.14s/it]


In [267]:
query = ["Children intelligence "]
query_embedding = model.encode(query).tolist()

In [268]:
# perform a basic search for top 4 results without document metadata filtering
print(f"Pinecone query without metadata:")

index.query(
  vector=query_embedding,
  top_k=4,
  include_values=False,
  include_metadata=True
)

Pinecone query without metadata:


{'matches': [{'id': '24083565_0',
              'metadata': {'article_id': '24083565',
                           'authior': ['Tanabe Mark K',
                                       'Whitaker Ashley M',
                                       "O'Callaghan Erin T",
                                       'Murray Joan',
                                       'Houskamp Beth M'],
                           'text': 'the purpose of this study was to explore '
                                   'the relationship between intelligence and '
                                   'executive functioning in youth. more '
                                   'specifically, the authors hypothesized '
                                   'that there would be a positive correlation '
                                   'between scores on the wechsler '
                                   'intelligence scale for children - fourth '
                                   'edition general ability index ( gai ) and '
   

In [269]:
#search with metadata filtering
print(f"\nPinecone query with author filter and metadata:")

index.query(
     vector=query_embedding,


     filter={'authior': {'$in': ['Tanabe Mark K']}},
     top_k=4,
     include_metadata=True # if true then metadata values are displayed
)



Pinecone query with author filter and metadata:


{'matches': [{'id': '24083565_0',
              'metadata': {'article_id': '24083565',
                           'authior': ['Tanabe Mark K',
                                       'Whitaker Ashley M',
                                       "O'Callaghan Erin T",
                                       'Murray Joan',
                                       'Houskamp Beth M'],
                           'text': 'the purpose of this study was to explore '
                                   'the relationship between intelligence and '
                                   'executive functioning in youth. more '
                                   'specifically, the authors hypothesized '
                                   'that there would be a positive correlation '
                                   'between scores on the wechsler '
                                   'intelligence scale for children - fourth '
                                   'edition general ability index ( gai ) and '
   